# PlayTimeGenre

Crear una función que tenga como entrada una cadena de texto representando un género y nos devuelva el año con más horas jugadas para dicho género. Para hacer esto se va a crear el CSV con la información necesaria para realizar la consulta, ya que se debe deployar en Render y los datasets son muy pesados para cargarlos.

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}


### Importamos las librerías

In [1]:
import pandas as pd
import ast

### Lectura del archivo CSV

In [2]:
steam_games = pd.read_csv("..\\Datsets\\CSV\\steam_games.csv")
user_items =pd.read_csv("..\\Datsets\\CSV\\user_items.csv")

### Creación del CSV para la función PlayTimeGenre

La información que necesitamos incluye los géneros, el año de lanzamiento y el tiempo jugado en total por cada juego. Para ello, vamos a utilizar las columnas 'id', 'genres', 'release_year' del conjunto de datos `steam_games`, y las columnas 'item_id' y 'playtime_forever' del conjunto de datos `user_items`. Vamos a concatenar los DataFrames a través de las columnas 'id' e 'item_id'.

1. **Columnas de steam_games:**
    - 'id'
    - 'genres'
    - 'release_year'

2. **Columnas de user_items:**
    - 'item_id'
    - 'playtime_forever'

Vamos a realizar la concatenación utilizando las columnas 'id' e 'item_id'.


In [6]:
# Combinar DataFrames utilizando las variables en español
tiempo_juego_genero = pd.merge(steam_games[['id', 'genres', 'release_year']],
                               user_items[['item_id', 'playtime_forever']],
                               left_on='id', right_on='item_id', how='inner')

# Eliminar las columnas duplicadas 'item_id' e 'id'
tiempo_juego_genero = tiempo_juego_genero.drop(columns=['item_id'], axis=1)


In [13]:
tiempo_juego_genero.head()

,id,genres,release_year,playtime_forever
0,282010.0,"['Racing', 'Action', 'Indie']",1997.0,5
1,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
2,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
3,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
4,282010.0,"['Racing', 'Action', 'Indie']",1997.0,13


Agrupar la columna 'playtime_forever' por cada 'id' para obtener el tiempo total jugado por cada juego

In [14]:
#Agrupamos por playtime_forever por id
tiempo_juego_genero = tiempo_juego_genero.groupby(['genres', 'release_year', 'id'])['playtime_forever'].sum().reset_index()

In [15]:
tiempo_juego_genero.head()

,genres,release_year,id,playtime_forever
0,"['Action', 'Adventure', 'Casual', 'Indie']",2016.0,499460.0,13
1,"['Action', 'Adventure', 'Casual', 'Indie']",2016.0,514890.0,2
2,"['Action', 'Adventure', 'Casual', 'Strategy']",2015.0,355020.0,0
3,"['Action', 'Adventure', 'Casual']",1999.0,20.0,960524
4,"['Action', 'Adventure', 'Casual']",2009.0,319170.0,5581


Expandimos las listas en la columna 'genres'

In [16]:
#Convertimos la columna genres a tipo lista
tiempo_juego_genero['genres'] = tiempo_juego_genero['genres'].apply(ast.literal_eval)
#Expandimos las listas en columnas
tiempo_juego_genero = tiempo_juego_genero.explode('genres').reset_index(drop=True)

In [17]:
tiempo_juego_genero.head()

,genres,release_year,id,playtime_forever
0,Action,2016.0,499460.0,13
1,Adventure,2016.0,499460.0,13
2,Casual,2016.0,499460.0,13
3,Indie,2016.0,499460.0,13
4,Action,2016.0,514890.0,2


Agrupamos por 'genres' y 'release_year', sumando las horas jugadas para cada combinación.

In [18]:
#Agrupamos por genero y año de lanzamiento
tiempo_juego_genero = tiempo_juego_genero.groupby(['release_year', 'genres'])['playtime_forever'].sum().reset_index()

In [19]:
tiempo_juego_genero.head()

,release_year,genres,playtime_forever
0,1983.0,Action,3473
1,1983.0,Adventure,3473
2,1983.0,Casual,3473
3,1984.0,Action,384
4,1984.0,Adventure,384


El resultado es un DataFrame que contiene la información de cuántas horas se han jugado por género y año de lanzamiento. Convertiremos este DataFrame a un archivo CSV para alimentar la API.

In [20]:
# Guardar el DataFrame en un archivo CSV
tiempo_juego_genero.to_csv('..\\Datsets\\Archivos_API\\PlayTimeGenre.csv', index=False)


Creación de la función para consultar el año con más horas jugadas por género

In [26]:
def PlayTimeGenre(genero):
    """
    Función que toma como entrada un género y devuelve el año con la máxima cantidad de horas jugadas para ese género.

    Parámetros:
    - genero (str): Género para el cual se busca el año con más horas jugadas.

    Ejemplo de retorno:
    - Para un género existente: 2013
    - Para un género no encontrado: "No se encontraron datos para el género 'Género X'"
    """
    # Filtra el DataFrame para el género especificado

    df_genero = tiempo_juego_genero[tiempo_juego_genero['genres'] == genero]

    # Verifica si el género está presente en el DataFrame
    if not df_genero.empty:
        # Encuentra el año con la máxima cantidad de horas jugadas
        anio_mas_jugado = df_genero.loc[df_genero['playtime_forever'].idxmax()]['release_year']
        return f'Año de lanzamiento con más horas jugadas para el género {genero} : {int(anio_mas_jugado)}'
    else:
        return f"No se encontraron datos para el género '{genero}'"
